# Lecture 12: Eigenvalues and Eigenvectors

## Power iteration

The iteration

$$
\mathbf{b}_{k+1} = \frac{A\mathbf{b}_k}{\| A\mathbf{b}_k \|}
$$

converges to an eigenvector of the largest eigenvalue of $A$ (as long as the starting
guess $\mathbf{b}_0$ had a component in that eigenspace, and the largest eigenvalue
is separate from the others).
The eigenvalue is then found from

$$
\mathbf{b}_k^\top A \mathbf{b}_k \rightarrow \lambda_{\text{max}}.
$$

In [2]:
using LinearAlgebra

function power_iterate(A, b; N=10000, ϵ=√eps())
    """ Compute the dominant eigenvalue of A
    using power iteration
    """
    for i=1:N
        # power iteration
        a = A*b
        b_new = a/norm(a)
        
        # Termination criterion
        if norm(b - b_new) < ϵ*norm(b_new)
            break
        end
        
        b = b_new
    end
    
    # return eigenvalue and eigenvector
    return b'*A*b, b
end

power_iterate (generic function with 1 method)

In [3]:
A = [1 3 4
4 5 0
2 1 3]

# Julia's internal function
eigvals(A)

3-element Array{Float64,1}:
 -1.8989794855663558
  7.898979485566356 
  3.0               

In [4]:
b0 = [1.0, 2.0, 3.0]

λ_max, v = power_iterate(A, b0)

(7.898979491079361, [0.543945, 0.750533, 0.375266])

In [5]:
# check that the eigenvector is correct
A*v ./ v

3-element Array{Float64,1}:
 7.898979436563664
 7.898979556654386
 7.898979343317884

## The QR algorithm
The QR algorithm (or rather, a significantly more sophisticated variant) 
is often used in practice (for example LAPACK,
which underlies Julia's eig function) to compute eigenvalues of dense matrices.

The QR algorithm was named one of the top 10 most influential algorithms
(https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=17639&punumber=5992).

We are interested in the eigenvalues of the square matrix $A$.
Set $A = Q_0 R_0$.

Then iterate 
   *  $A_{k+1} = R_k Q_k$ (reverse order of $QR$ factors and multiply)
   *  $A_{k+1} = Q_{k+1} R_{k+1}$ (QR decomposition)
   
All the iterates are similar matrices:

$$
A_{k+1} = R_k Q_k = Q_k^\top A_{k} Q_k,
$$

so the must have the same eigenvalues.

Here's the magic:
under some conditions, this iteration will converge to a *triangular* matrix,
whose eigenvalues can be read off from the diagonal!

In [6]:
function eig_qr(A; N=1000, ϵ=√eps())
    """ Compute eigenvalues of A using the QR algorithm
    """
    F = qr(A)
    Q, R = F.Q, F.R
    
    for i=1:N
        # multiply in "wrong" order
        A_new = R*Q
        
        # termination criterion
        if norm(A_new - A) < ϵ*norm(A_new)
            break
        end
        
        A = A_new
        
        # QR decomposition
        F = qr(A)
        Q, R = F.Q, F.R
    end
    
    return A
end

eig_qr (generic function with 1 method)

In [7]:
A = [1 3 4
4 5 0
2 1 3]

# Julia's internal function
eigvals(A)

3-element Array{Float64,1}:
 -1.8989794855663558
  7.898979485566356 
  3.0               

In [11]:
# QR algorithm
eig_qr(A; N=100)

3×3 Array{Float64,2}:
  7.89898      -0.338159      0.167063
 -3.01662e-58   3.0          -2.42028 
  1.68621e-61  -1.66516e-19  -1.89898 

## What is this magic?!
How does the QR algorithm work?
It's just power iteration in disguise!

Recall the power iteration:
$$
\mathbf{b}_k = \frac{A\mathbf{b}_{k-1}}{\| A\mathbf{b}_{k-1} \|} = \frac{A^k \mathbf{b}_0}{\| A^k \mathbf{b}_0 \|}.
$$

Power iteration converges to the dominant eigenvalue _if_ the start vector has a component in
that eigenspace.

Intuitively, we want to compute power iterates of $n$ initial guesses (the unit vectors $\mathbb{1}$), 
and then orthonormalize them. This way, if one of the vectors converges to the dominant
eigenvector, all the others are orthogonal to it and have no component in that eigenspace.
So, they must converge to different eigenvectors, and we will see that this is what the QR algorithm does.

We first formalize this idea in the algorithm of Orthogonal Iteration.

### Orthogonal iteration
Each step we iterate $A$ and then orthonormalize using the QR decomposition:

$$
\hat Q_{k+1} R_{k+1} = A \hat Q_{k}, \qquad \hat Q_0 R_0 = A.
$$

To see what this iteration does, let's compute the first step:
$$
\hat Q_0 R_0 = A
$$
and the second step:
$$
\hat Q_1 R_1 = A \hat Q_0.
$$

Multiply by $R_0$ on the right,
$$
\hat Q_1 R_1 R_0 = A \hat Q_0 R_0 = A A = A^2.
$$

Continue this and find
$$
\hat Q_{k-1} \underbrace{ R_{k-1} R_{k-2} \cdots R_0}_{=\hat R_{k-1}} = \hat Q_{k-1} \hat R_{k-1} =  A^k,
$$

so effectively we are computing factors of a QR decomposition of $A^k$ (products of triangular
matrices are again triangular).
(Recall that we did derive this iteration from the power method).

By construction, the matrix $\hat Q_k$ converges to a set of orthonormal eigenvectors of $A$
(if it converges, and if $A$ has orthonormal eigenvectors. For instance, symmetric $A$ have orthonormal
eigenvectors).

### From orthogonal to QR iteration
The QR algorithm is just a different way of computing the exact same
decomposition of $A^k$:

\begin{align}
A &= Q_0 R_0 \\
A^2 &= Q_0 \underbrace{R_0 Q_0}_{=A_1 = Q_1 R_1} R_0 \\
\vdots \\
\Rightarrow A^k &= \left(Q_0 Q_1 \cdots Q_{k-1}\right) \left( R_{k-1} \cdots R_1 R_0 \right).
\end{align}

So, the product $Q_0 Q_1 \cdots Q_{k} = \hat Q_{k}$, and $ R_{k} \cdots R_1 = \hat R_k$.


### Intuitive "proof" of convergence to an upper triangular matrix:
If the QR algorithm converges, then $A_k = R_{k-1} Q_{k-1}$ converges to a constant.

From our analysis of the Orthogonal Iteration, we should expect $Q_k$ to converge to $Q_k \approx \mathbb{1}$,
because the product $\hat Q_k = Q_0 Q_1 \cdots Q_k$ converges to a constant matrix of eigenvectors.

But from this we get,
$$
A_k \approx R_k,
$$
so it converges to an upper triangular matrix.
Since this matrix is similar to the original $A$, it will have the eigenvalues of $A$ on its diagonal.

There are rigorous (but complicated) proofs for this available under certain technical assumptions (see for example Suli & Meyers).

In [20]:
# sometimes the simple QR algorithm fails
A = [ 0 1.0
1 0]

# Oops
eig_qr(A; N=4)

2×2 Array{Float64,2}:
 0.0  1.0
 1.0  0.0

In [21]:
# Julia's algorithm is more sophisticated!

# "Implicit QR algorithm with Wilkinson shifts"
eigvals(A)

2-element Array{Float64,1}:
 -1.0
  1.0